# Import necessary libraries

In [ ]:
import cv2
import torch
import torchvision
import scipy
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

# Utilities function

In [ ]:
def place_image(bg,
                card,
                x = None,
                y = None,
                rotate = 0,
                card_scale=0.3):
    # copy
    bg = bg.copy()
    card = card.copy()
    
    # resize card
    card = cv2.resize(card, (int(card.shape[1] * card_scale), int(card.shape[0] * card_scale)))
    if card.shape[0] > bg.shape[0] or card.shape[1] > bg.shape[1]:
        card_scale = min(bg.shape[0] / card.shape[0], bg.shape[1] / card.shape[1])
        card = cv2.resize(card, (int(card.shape[1] * card_scale), int(card.shape[0] * card_scale)))
        
    
    # rotate card
    if rotate != 0:
        card = scipy.ndimage.rotate(card, rotate, reshape=True)

    # set x, y if none -> random
    try:
        if x is None:
            x = random.randint(0, bg.shape[1] - card.shape[1])
        if y is None:
            y = random.randint(0, bg.shape[0] - card.shape[0])
    except:
        x = 0
        y = 0
    # place card avoid background of rotated card and avoid overflow
    try:
        if rotate != 0:
            bg[y:y+card.shape[0], x:x+card.shape[1]] = np.where(card != 0, card, bg[y:y+card.shape[0], x:x+card.shape[1]])
        else:
            bg[y:y+card.shape[0], x:x+card.shape[1]] = card
    except:
        pass    
    # blur image to make it more realistic
    bg = cv2.GaussianBlur(bg, (5, 5), 0)

    # histogram equalization
    bg = cv2.cvtColor(bg, cv2.COLOR_BGR2HSV)
    bg[:, :, 2] = cv2.equalizeHist(bg[:, :, 2])
    bg = cv2.cvtColor(bg, cv2.COLOR_HSV2BGR)

    # bounding block
    bounding_block = {
        'x': x,
        'y': y,
        'width': card.shape[1],
        'height': card.shape[0],
        'rotate': rotate,
    }

    return bg, bounding_block

In [ ]:
def extract_annotation(xml_path):
    # load annotation from .xml file
    with open(xml_path) as f:
        xml = f.read()

    # parse xml
    root = ET.fromstring(xml)
    annotation = {}
    for child in root:
        if child.tag == 'size':
            annotation['size'] = {}
            for size in child:
                annotation['size'][size.tag] = int(size.text if size.tag != 'depth' else 0)
        elif child.tag == 'object':
            if 'object' not in annotation:
                annotation['object'] = []
            obj = {}
            for obj_child in child:
                if obj_child.tag == 'name':
                    obj['name'] = obj_child.text
                elif obj_child.tag == 'bndbox':
                    obj['bndbox'] = {}
                    for bndbox in obj_child:
                        obj['bndbox'][bndbox.tag] = float(bndbox.text)
            annotation['object'].append(obj)
    
    return annotation

In [ ]:
def place_card_to_bb(original_img,
                     card,
                     bounding_block):
    '''
    bounding block must in this structure
    {
        'x': x,
        'y': y,
        'width': card.shape[1],
        'height': card.shape[0],
        'rotate': rotate,
    }
    '''
    # copy
    bg = original_img.copy()
    card = card.copy()

    # resize card to bounding block
    card = cv2.resize(card, (int(bounding_block['width']), int(bounding_block['height'])))
    
    # place card
    x = int(bounding_block['x'])
    y = int(bounding_block['y'])
    bg[y:y+card.shape[0], x:x+card.shape[1]] = np.where(card != 0, card, bg[y:y+card.shape[0], x:x+card.shape[1]])

    return bg
    

# Test : Place card to the background image

In [ ]:
TARGET_SIZE = (512, 512)

BG = cv2.imread('./asset/bg/1624549127.jpg')
BG = cv2.resize(BG, TARGET_SIZE)
CARD = cv2.imread('./asset/card/pii_card1.jpg')


plt.subplot(1, 2, 1)
plt.imshow(BG)
plt.subplot(1, 2, 2)
plt.imshow(CARD)
plt.show()

In [ ]:
plt.subplot(1, 2, 1)
plt.title('BG')
plt.imshow(BG)

plt.subplot(1, 2, 2)
plt.title('place_image')
plt.imshow(place_image(BG.copy(), CARD, rotate=90)[0])

plt.show()

In [ ]:
for bg_name in os.listdir('./asset/bg'):
    bg = cv2.imread(os.path.join('./asset/bg', bg_name))
    bg = cv2.cvtColor(bg, cv2.COLOR_BGR2RGB)
    bg = cv2.resize(bg, TARGET_SIZE)
    for card in os.listdir('./asset/card'):
        if random.random() > 0.5:
            continue
        card = cv2.imread(os.path.join('./asset/card', card))
        card = cv2.cvtColor(card, cv2.COLOR_BGR2RGB)

        rotate = np.random.normal(0, 10)
        scale = np.random.uniform(0.1, 0.3)
        replaced_bg, bounding_block = place_image(bg, card, rotate=rotate, )

        plt.subplot(1, 3, 1)
        plt.title('BG')
        plt.imshow(bg)

        plt.subplot(1, 3, 2)
        plt.title('place_image')
        plt.imshow(replaced_bg)

        plt.subplot(1, 3, 3)
        plt.title('bounding_block')
        plt.imshow(replaced_bg)
        plt.gca().add_patch(plt.Rectangle((bounding_block['x'], bounding_block['y']), bounding_block['width'], bounding_block['height'], linewidth=1, edgecolor='r', facecolor='none'))

        plt.show()


# Test : Place our card to replace the original card image

In [ ]:
file_name = '20210611_21_03_24_000_2sQ7uoDSHQakPcTTKy9Ikfs5wWs1_F_3264_2448'
# load image
img = cv2.imread(f'./asset/visiting_card/id_card/{file_name}.jpg')

# load annotation
annotation = extract_annotation(f'./asset/visiting_card/annotation/annotation/{file_name}.xml')

# draw bounding box
for obj in annotation['object']:
    img = cv2.rectangle(img, (int(obj['bndbox']['xmin']), int(obj['bndbox']['ymin'])), (int(obj['bndbox']['xmax']), int(obj['bndbox']['ymax'])), (0, 255, 0), 10)

plt.subplot(1, 2, 1)
plt.imshow(img)

# load card
card = cv2.imread('./asset/card/pii_card1.jpg')

# place card to bounding box
img = place_card_to_bb(img, card, {
    'x' : annotation['object'][0]['bndbox']['xmin'],
    'y' : annotation['object'][0]['bndbox']['ymin'],
    'width' : annotation['object'][0]['bndbox']['xmax'] - annotation['object'][0]['bndbox']['xmin'],
    'height' : annotation['object'][0]['bndbox']['ymax'] - annotation['object'][0]['bndbox']['ymin'],
})

plt.subplot(1, 2, 2)
plt.imshow(img)

plt.show()



# Generate the final image dataset

- dataset structure
```
dataset
├── labels
│   ├── 0.txt
├── images
    ├── 0.jpg
```



## Config the environment

In [ ]:
LABEL_FOLDER = './dataset/labels'
IMAGE_FOLDER = './dataset/images'

TARGET_SIZE = (512, 512)

In [ ]:
# create if not exist and clear folder
for folder in [LABEL_FOLDER, IMAGE_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)
    else:
        for file in os.listdir(folder):
            os.remove(os.path.join(folder, file))


## Place card to the background image

In [ ]:
for bg_file in os.listdir('./asset/bg'):
    if not bg_file.endswith('.jpg'):
        continue
    bg = cv2.imread(os.path.join('./asset/bg', bg_file))
    bg = cv2.cvtColor(bg, cv2.COLOR_BGR2RGB)
    bg = cv2.resize(bg, TARGET_SIZE)

    looper = tqdm(os.listdir('./asset/card'), desc=bg_file)
    for card_file in looper:
        if not card_file.endswith('.jpg'):
            pass
        card = cv2.imread(os.path.join('./asset/card', card_file))
        card = cv2.cvtColor(card, cv2.COLOR_BGR2RGB)
    
        rotate = np.random.normal(0, 10)
        card_scale = np.random.uniform(0.1, 0.4)
        placed_image, bounding_block = place_image(bg, card, rotate=rotate, card_scale=card_scale)

        # crate label file
        label_file = f"{bg_file.split('.')[0]}_{card_file.split('.')[0]}.txt"
        with open(os.path.join(LABEL_FOLDER, label_file), 'w') as f:
            label = 1 if 'pii' in card_file else 0
            x_cent = (bounding_block['x'] + bounding_block['width'] / 2) / bg.shape[1]
            y_cent = (bounding_block['y'] + bounding_block['height'] / 2) / bg.shape[0]
            width = bounding_block['width'] / bg.shape[1]
            height = bounding_block['height'] / bg.shape[0]
            f.write(f'{label} {x_cent} {y_cent} {width} {height}')

        # save image
        cv2.imwrite(os.path.join(IMAGE_FOLDER, label_file.replace('.txt', '.jpg')), cv2.cvtColor(placed_image, cv2.COLOR_RGB2BGR))


### Validate data set

In [ ]:
test_file = '664x38620_enhanced_normal_card1'

# load image
img = cv2.imread(f'./dataset/images/{test_file}.jpg')

# load annotation
with open(f'./dataset/labels/{test_file}.txt') as f:
    label = f.read()
    label, x_cent, y_cent, width, height = label.split(' ')
    x_cent = float(x_cent)
    y_cent = float(y_cent)
    width = float(width)
    height = float(height)

# draw bounding box
img = cv2.rectangle(img, (int((x_cent - width / 2) * img.shape[1]), int((y_cent - height / 2) * img.shape[0])), (int((x_cent + width / 2) * img.shape[1]), int((y_cent + height / 2) * img.shape[0])), (0, 255, 0), 5)

plt.imshow(img)
plt.show()



# Place our card to replace the original card image

In [ ]:
looper = tqdm(os.listdir('./asset/visiting_card/id_card'), desc='background image')
for bg_file in looper:
    if not bg_file.endswith('.jpg'):
        continue
    
    # load image
    bg = cv2.imread(os.path.join('./asset/visiting_card/id_card', bg_file))
    bg = cv2.cvtColor(bg, cv2.COLOR_BGR2RGB)

    # load annotation
    annotation = extract_annotation(os.path.join('./asset/visiting_card/annotation/annotation', bg_file.replace('.jpg', '.xml')))
    if 'object' not in annotation:
        continue
    bounding_block = {
        'x': int(annotation['object'][0]['bndbox']['xmin']),
        'y': int(annotation['object'][0]['bndbox']['ymin']),
        'width': int(annotation['object'][0]['bndbox']['xmax'] - annotation['object'][0]['bndbox']['xmin']),
        'height': int(annotation['object'][0]['bndbox']['ymax'] - annotation['object'][0]['bndbox']['ymin']),
    }

    for card_file in os.listdir('./asset/card'):
        if not card_file.endswith('.jpg'):
            continue
        card = cv2.imread(os.path.join('./asset/card', card_file))
        card = cv2.cvtColor(card, cv2.COLOR_BGR2RGB)

        # resize card to bounding block
        card = cv2.resize(card, (int(bounding_block['width']), int(bounding_block['height'])))

        # place card
        replaced_bg = place_card_to_bb(bg, card, bounding_block)
        
        # resize image
        replaced_bg = cv2.resize(replaced_bg, TARGET_SIZE)
        
        # save image
        cv2.imwrite(os.path.join(IMAGE_FOLDER, f"{bg_file.replace('.jpg', '')}_{card_file.replace('.jpg', '')}.jpg"), cv2.cvtColor(replaced_bg, cv2.COLOR_RGB2BGR))
        plt.imshow(replaced_bg)

        # crate label file
        label_file = f"{bg_file.replace('.jpg', '')}_{card_file.replace('.jpg', '')}.txt"
        with open(os.path.join(LABEL_FOLDER, label_file), 'w') as f:
            label = 1 if 'pii' in card_file else 0
            x_cent = (bounding_block['x'] + bounding_block['width'] / 2) / bg.shape[1]
            y_cent = (bounding_block['y'] + bounding_block['height'] / 2) / bg.shape[0]
            width = bounding_block['width'] / bg.shape[1]
            height = bounding_block['height'] / bg.shape[0]
            f.write(f'{label} {x_cent} {y_cent} {width} {height}')


## Validate data set

In [ ]:
file_name = '20210613_11_06_09_000_0jKd39aZFrUJQhjwr4kWJ5lPMMF3_F_4160_3120_normal_card1'
# load image
image = cv2.imread(f'./dataset/images/{file_name}.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# load label
with open(f'./dataset/labels/{file_name}.txt') as f:
    label = f.read()
    label, x_cent, y_cent, width, height = label.split(' ')
    x_cent = float(x_cent)
    y_cent = float(y_cent)
    width = float(width)
    height = float(height)

# draw bounding box
image = cv2.rectangle(image, (int((x_cent - width / 2) * image.shape[1]), int((y_cent - height / 2) * image.shape[0])), (int((x_cent + width / 2) * image.shape[1]), int((y_cent + height / 2) * image.shape[0])), (0, 255, 0), 5)

plt.imshow(image)
plt.show()


## No card image

In [ ]:
looper = tqdm(os.listdir('./asset/bg/'), desc='background image')
for bg_file in looper:
    if not bg_file.endswith('.jpg'):
        continue
    bg = cv2.imread(os.path.join('./asset/bg', bg_file))
    bg = cv2.cvtColor(bg, cv2.COLOR_BGR2RGB)
    bg = cv2.resize(bg, TARGET_SIZE)

    # save image
    cv2.imwrite(os.path.join(IMAGE_FOLDER, f"{bg_file.replace('.jpg', '')}.jpg"), cv2.cvtColor(bg, cv2.COLOR_RGB2BGR))

    # crate label file
    label_file = f"{bg_file.replace('.jpg', '')}.txt"
    with open(os.path.join(LABEL_FOLDER, label_file), 'w') as f:
        f.write('')


# Dataset summary

In [ ]:
def show_dict(d, t = ''):
    for key in d:
        print(t + key, end = ' ')
        if type(d[key]) == dict:
            show_dict(d[key], t + '\t')
        else:
            print(':' + str(d[key]))

In [ ]:
info = {
    'class' : {
        '0' : 0,
        '1' : 0,
    },
}
for label in os.listdir('./dataset/labels'):
    with open(os.path.join('./dataset/labels', label)) as f:
        label = f.read()
        label, x_cent, y_cent, width, height = label.split(' ')
        x_cent = float(x_cent)
        y_cent = float(y_cent)
        width = float(width)
        height = float(height)
        
        info['class'][label] += 1

show_dict(info)